Hybrid Search Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
api_key = os.getenv("PINECODE_API_KEY")

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone, ServerlessSpec

index_name = "hybrid-search-langchain-pinecone"

# initialize the Pinecone client
pc = Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,   # dimension of dense vector
        metric= "dotproduct",  # sparse values supported only for the dotproduct
        spec = ServerlessSpec(cloud="aws",region="us-east-1")

    )

In [4]:
index = pc.Index(index_name)
index

/home/devendra/Desktop/Udemy_Generative_AI_With_Langchain_and_Huggingface_2024/Project/Langchain_Projects/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# vector embedding and sparse matrix

from langchain_huggingface import HuggingFaceEmbeddings

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings

/home/devendra/Desktop/Udemy_Generative_AI_With_Langchain_and_Huggingface_2024/Project/Langchain_Projects/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
from pinecone_text.sparse import BM25Encoder    # BM25Encoder uses TF-IDF technique for sparse matrixs conversartion

bm25_encoder = BM25Encoder.default()   #default is TF-IDF
bm25_encoder

In [7]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York ",
    "In 2021, I visited New Delhi"
]


#applying  tfidf values in these sentences
bm25_encoder.fit(sentences)

#store the values to a json file
bm25_encoder.dump("bm25_values.json")


#load to em25encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 142.11it/s]


In [8]:
# create retriever

retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder,index=index)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x77e8b0276680>, index=<pinecone.db_data.index.Index object at 0x77e8b0277220>)

In [9]:
retriever.add_texts(sentences)

100%|██████████| 1/1 [00:02<00:00,  2.28s/it]


In [10]:
retriever.invoke("What city did i visited last?")

[Document(metadata={'score': 0.270836}, page_content='In 2022, I visited New York '),
 Document(metadata={'score': 0.250163287}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.236216679}, page_content='In 2021, I visited New Delhi')]

In [ ]:
retriever.invoke("When i visited New Delhi?") 

[Document(metadata={'score': 0.643733144}, page_content='In 2021, I visited New Delhi'),
 Document(metadata={'score': 0.327555031}, page_content='In 2022, I visited New York '),
 Document(metadata={'score': 0.237566143}, page_content='In 2023, I visited Paris')]